In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from rdflib import Graph, Namespace, Literal, XSD, RDF, URIRef
from NRI_to_RDF import *
ace = Namespace("http://www.semanticweb.org/XKG#")

<Graph identifier=N60637b0845374c198273712e5908b5f5 (<class 'rdflib.graph.Graph'>)>

http://www.semanticweb.org/XKG#12
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.semanticweb.org/XKG#Author
